### Problem 1

Load the Movielens 100k dataset (ml-100k.zip) into Python using Pandas
dataframes. Build a user profile on centered data (by user rating) for both users
200 and 15, and calculate the cosine similarity and distance between the user’s
preferences and the item/movie 95. Which user would a recommender system
suggest this movie to?

In [1]:
import numpy as np
import pandas as pd

from sklearn import metrics

In [2]:
user_cols = ['user_id',
             'age',
             'gender',
             'occupation',
             'zip_code']

users = pd.read_csv('/content/u.user',
                    sep='|',
                    names=user_cols)

In [3]:
users.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
item_cols = ['movie id',
             'movie title',
             'release date',
             'video release date',
             'IMDb URL',
             'Unknown',
             'Action',
             'Adventure',
             'Animation',
             'Childrens',
             'Comedy',
             'Crime',
             'Documentary',
             'Drama',
             'Fantasy',
             'FilmNoir',
             'Horror',
             'Musical',
             'Mystery',
             'Romance',
             'SciFi',
             'Thriller',
             'War',
             'Western']

items = pd.read_csv('/content/u.item',
                    sep='|',
                    names=item_cols,
                    encoding='latin-1')

In [5]:
items.head()

,movie id,movie title,release date,video release date,IMDb URL,Unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [6]:
rating_cols = ['user_id',
               'movie_id',
               'rating',
               'timestamp']

ratings = pd.read_csv('/content/u.data',
                      sep='\t',
                      names=rating_cols)

In [7]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Building a utility matrix.

In [8]:
utility = ratings.pivot(index='user_id',
                        columns='movie_id',
                        values='rating')

utility.head()

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
user_means = utility.mean(axis=1)
user_means.head()

user_id
1    3.610294
2    3.709677
3    2.796296
4    4.333333
5    2.874286
dtype: float64

Centering the utility matrix.

In [10]:
utility_centered = utility - user_means
utility_centered = utility_centered.where((pd.notnull(utility_centered)),0)

utility_centered.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.389706,-0.709677,1.203704,-1.333333,0.125714,1.364929,0.034739,-2.79661,0.727273,-1.206522,-1.464088,0.607843,1.902516,0.908163,2.125,0.671429,-0.035714,0.119134,1.45,0.895833,-1.670391,0.648438,0.364238,-1.323529,-0.051282,0.056075,-1.24,0.278481,-2.647059,-0.767442,-0.916667,1.682927,0.291667,-2.05,-2.0,-1.8,-1.649123,-0.719008,0.227273,0.114286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.389706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,-2.206522,0.000000,0.000000,0.902516,-0.091837,0.000,0.000000,0.000000,0.000000,-0.55,0.000000,0.000000,0.000000,0.000000,0.000000,-0.051282,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.535912,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.389706,-0.709677,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.964286,0.000000,0.00,0.000000,0.329609,0.000000,0.000000,-0.323529,-1.051282,0.000000,0.00,0.000000,0.352941,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,1.114286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
data = {'user_id':[15, 200],
        'Unknown':[0,0],
        'Action':[0,0],
        'Adventure':[0,0],
        'Animation':[0,0],
        'Childrens':[0,0],
        'Comedy':[0,0],
        'Crime':[0,0],
        'Documentary':[0,0],
        'Drama':[0,0],
        'Fantasy':[0,0],
        'FilmNoir':[0,0],
        'Horror':[0,0],
        'Musical':[0,0],
        'Mystery':[0,0],
        'Romance':[0,0],
        'SciFi':[0,0],
        'Thriller':[0,0],
        'War':[0,0],
        'Western':[0,0]}
  
# Create DataFrame
user_profiles = pd.DataFrame(data)

In [12]:
user_profiles.head()

,user_id,Unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
data = {'user_id':[15, 200],
        'Unknown':[0,0],
        'Action':[0,0],
        'Adventure':[0,0],
        'Animation':[0,0],
        'Childrens':[0,0],
        'Comedy':[0,0],
        'Crime':[0,0],
        'Documentary':[0,0],
        'Drama':[0,0],
        'Fantasy':[0,0],
        'FilmNoir':[0,0],
        'Horror':[0,0],
        'Musical':[0,0],
        'Mystery':[0,0],
        'Romance':[0,0],
        'SciFi':[0,0],
        'Thriller':[0,0],
        'War':[0,0],
        'Western':[0,0]}
  
# Create DataFrame
counts = pd.DataFrame(data)

Creating user profiles for users 200 and 15, by taking the mean of the centered ratings for each feature. 

In [14]:
for i in range(1682):
  user_profiles.iloc[0, 1:] = user_profiles.iloc[0, 1:] + (utility_centered.iloc[14, i])*items.iloc[i , 5:]
  counts.iloc[0, :] = counts.iloc[0, :] + items.iloc[i , 5:]
user_profiles.iloc[0, 1:] = user_profiles.iloc[0, 1:]/counts.iloc[0, :]

In [15]:
for i in range(1682):
  user_profiles.iloc[1, 1:] = user_profiles.iloc[1, 1:] + (utility_centered.iloc[199, i])*items.iloc[i , 5:]
  counts.iloc[1, :] = counts.iloc[1, :] + items.iloc[i , 5:]
user_profiles.iloc[1, 1:] = user_profiles.iloc[1, 1:]/counts.iloc[1, :]

In [16]:
user_profiles

,user_id,Unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,15,0.0,-0.072787,-0.041764,-0.131538,-0.115606,-0.051175,-0.057916,0.000000,-0.033429,-0.080099,-0.018082,-0.070458,-0.050718,-0.101594,-0.007098,-0.103562,-0.124119,-0.004673,0.000000
1,200,0.0,0.213896,0.262589,0.182118,0.116114,0.017444,0.001876,-0.035758,0.047255,0.166779,0.086451,0.092176,0.155684,0.040464,0.102339,0.363491,0.089724,0.190770,0.073511


Calculating cosine similarity and distance.

In [17]:
print("Cosine similarity between user profile" , user_profiles.iloc[0, 0], "and movie", items.iloc[94, 0],":", metrics.pairwise.cosine_similarity(user_profiles.iloc[0:1, 1:], items.iloc[95:96, 5:]))
print("Cosine similarity between user profile", user_profiles.iloc[1, 0], "and movie", items.iloc[94, 0],":", metrics.pairwise.cosine_similarity(user_profiles.iloc[1:2, 1:], items.iloc[95:96, 5:]))
print("Cosine distance between user profile" , user_profiles.iloc[0, 0], "and movie", items.iloc[94, 0],":", 1 - metrics.pairwise.cosine_similarity(user_profiles.iloc[0:1, 1:], items.iloc[95:96, 5:]))
print("Cosine distance between user profile", user_profiles.iloc[1, 0], "and movie", items.iloc[94, 0],":", 1 - metrics.pairwise.cosine_similarity(user_profiles.iloc[1:2, 1:], items.iloc[95:96, 5:]))

Cosine similarity between user profile 15 and movie 95 : [[-0.56104962]]
Cosine similarity between user profile 200 and movie 95 : [[0.58922461]]
Cosine distance between user profile 15 and movie 95 : [[1.56104962]]
Cosine distance between user profile 200 and movie 95 : [[0.41077539]]


Calculating cosine similarity and distance after replacing negative centered ratings with 0 and positive with 1. 

In [18]:
user_profiles[user_profiles.iloc[:, 1: ] < 0] = 0
user_profiles[user_profiles.iloc[:, 1: ] > 0] = 1

In [19]:
user_profiles

,user_id,Unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,200,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [20]:
print("Cosine similarity between user profile" , user_profiles.iloc[0, 0], "and movie", items.iloc[94, 0],":", metrics.pairwise.cosine_similarity(user_profiles.iloc[0:1, 1:], items.iloc[95:96, 5:]))
print("Cosine similarity between user profile", user_profiles.iloc[1, 0], "and movie", items.iloc[94, 0],":", metrics.pairwise.cosine_similarity(user_profiles.iloc[1:2, 1:], items.iloc[95:96, 5:]))
print("Cosine distance between user profile" , user_profiles.iloc[0, 0], "and movie", items.iloc[94, 0],":", 1 - metrics.pairwise.cosine_similarity(user_profiles.iloc[0:1, 1:], items.iloc[95:96, 5:]))
print("Cosine distance between user profile", user_profiles.iloc[1, 0], "and movie", items.iloc[94, 0],":", 1 - metrics.pairwise.cosine_similarity(user_profiles.iloc[1:2, 1:], items.iloc[95:96, 5:]))

Cosine similarity between user profile 15 and movie 95 : [[0.]]
Cosine similarity between user profile 200 and movie 95 : [[0.42008403]]
Cosine distance between user profile 15 and movie 95 : [[1.]]
Cosine distance between user profile 200 and movie 95 : [[0.57991597]]


The recommender system will recommend movie 95 to user 200, because its cosine similarity is high and cosine distance is low.

### Problem 2

Load the Movielens 100k dataset (ml-100k.zip) into Python using Pandas
dataframes. Convert the ratings data into a utility matrix representation, and
find the 10 most similar users for user 1 based on cosine similarity of the centered
user ratings data. Based on the average of of the ratings for item 508 from the
similar users, what is the expected rating for this item for user 1?

The utility matrix:


In [21]:
utility = utility.where((pd.notnull(utility)),0)

In [22]:
utility.head()

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finding the top 10 most similar users to user 1.

In [23]:
similar_users = {}
for i in range(1,943):
  similar_users[i] = metrics.pairwise.cosine_similarity(utility_centered.iloc[0:1, 1:], utility_centered.iloc[i:i+1, 1:])

In [24]:
top_10 = sorted(similar_users, key=similar_users.get, reverse=True)[:10]

In [25]:
for i in top_10:
  print(i, similar_users[i])

737 [[0.28642353]]
591 [[0.27801816]]
275 [[0.26528778]]
266 [[0.26528436]]
456 [[0.26154415]]
756 [[0.26084901]]
642 [[0.2591531]]
605 [[0.25688873]]
915 [[0.25472145]]
43 [[0.25179566]]


Computing the expected rating.

In [26]:
sum = 0
counts = 0
for i in top_10:
  j = utility_centered.iloc[i-1, 507]
  if (j != 0):
    sum = sum + j
    counts += 1
avg = sum/counts
print(avg)

0.4224137931034484


In [27]:
user_means.iloc[0,] + avg

4.032707910750507

The expected rating for item 508 by user 1 is 4.032707910750507, meaning 4.